# Imports

In [1]:
import pandas as pd
import numpy as np
import re
import random
from typing import TypeVar, Tuple, Union
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.metrics import precision_score
import tensorflow as tf

import optuna

Self = TypeVar("Self")

2024-07-05 13:37:29.932466: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 13:37:29.932580: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 13:37:30.055888: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Chargement des datasets

In [ ]:
%%bash
if [ ! -d "./data/movielens_complete" ]; 
then    
    wget http://files.grouplens.org/datasets/movielens/ml-1m.zip 
    mkdir -p ./data/movielens_complete
    unzip -o ml-1m.zip -d ./data/movielens_complete;
else
    echo "Data already downloaded";
fi

In [ ]:
%%bash
if [ ! -d "./data/IMDb" ]; 
then    
    wget https://datasets.imdbws.com/title.basics.tsv.gz
    wget https://datasets.imdbws.com/title.crew.tsv.gz
    wget https://datasets.imdbws.com/title.ratings.tsv.gz
    mkdir -p ./data/IMDb
    gzip -dc title.basics.tsv.gz > ./data/IMDb/title.basics.tsv
    gzip -dc title.crew.tsv.gz > ./data/IMDb/title.crew.tsv
    gzip -dc title.ratings.tsv.gz > ./data/IMDb/title.ratings.tsv
else
    echo "Data already downloaded";
fi

# Fonctions utiles

In [2]:
def extract_year_and_title(row):
    """
    Fonction qui extrait l'année d'une chaine de caractères.
    Cela permet de séparer l'année et le titre d'un film
    
    Paramètres:
    ----------
    row: string
        la phrase contenant le titre et l'année de sortie
    
    Retours:
    -------
    Tuple[String, String]
        le titre du film et son année
    """
    match = re.search(r'\((\d{4})\)$', row)
    if match:
        year = match.group(1)
        title = row[:match.start()].strip()
    else:
        year = None
        title = row
    return title, year


def combine_genres(row):
    """
    Fonction pour combiner les genres des deux colonnes en une chaine de caractère (sans les dupliqués)
    
    Paramètres:
    ----------
    row: dataframe pandas
        la ligne du dataframe qui contient les deux colonnes (genre_x et genre_y)
    
    Retours:
    -------
    String
        l'ensemble des genres séparés par des ','
    """
    genrex = row['genres_x']
    genrey = row['genres_y']
    genres = genrex.split('|') + (genrey.split(',') if pd.notna(row['genres_y']) else [])
    #enlever les doublons
    unique_genres = list(set(genres))
    #recréer une string séparée par des ','
    return ','.join(unique_genres)


def split_ratings(
    ratings: np.ndarray, 
    val_size: Union[float, int], 
    test_size: Union[float, int]
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Fonction pour créer des ensembles de validation et de test à partir d'une matrice de notes

    Paramètres:
    ----------
    ratings : np.ndarray
        La matrice des notes (peut être clairsemée, remplie de zéros ou de valeurs manquantes).
    val_size : float ou int
        La taille de l'ensemble de validation.
        Si float, la taille est exprimée en pourcentage de la taille totale du tableau des notes.
        Si int, la taille est exprimée en nombre brut de notes dans l'ensemble de validation.
    test_size : float ou int
        La taille de l'ensemble de test.
        Si float, la taille est exprimée en pourcentage de la taille totale du tableau des notes.
        Si int, la taille est exprimée en nombre brut de notes dans l'ensemble de test.

    Retours:
    -------
    Tuple[np.ndarray, np.ndarray, np.ndarray]
        Le tuple des notes d'entraînement, de validation et de test.
    """
    #Prépare les 3 matrices de résultats
    rows, cols = np.where(ratings > 0)
    val_ratings = np.zeros(ratings.shape)
    test_ratings = np.zeros(ratings.shape)
    train_ratings = ratings.copy()

    total_ratings = len(rows)

    if val_size < 1:  # si c'est un pourcentage
        val_size = int(total_ratings * val_size)
    if test_size < 1:  # si c'est un pourcentage
        test_size = int(total_ratings * test_size)

    total_sample_size = val_size + test_size

    # Récupère aléatoirement les indices pour le set de validation et de test
    sample_indices = random.sample(range(total_ratings), total_sample_size)
    val_indices = sample_indices[:val_size]
    test_indices = sample_indices[val_size:]

    # Remplis le set de validation
    for idx in val_indices:
        row, col = rows[idx], cols[idx]
        #permet de les enlever du set d'entraînement pour les séparer totalement
        train_ratings[row, col] = 0
        val_ratings[row, col] = ratings[row, col]

    # Remplis le set de test
    for idx in test_indices:
        row, col = rows[idx], cols[idx]
        #permet de les enlever du set d'entraînement pour les séparer totalement
        train_ratings[row, col] = 0
        test_ratings[row, col] = ratings[row, col]

    return train_ratings, val_ratings, test_ratings


def normalizeRatings(Y: np.ndarray, R: np.ndarray, axis: int = 1) -> Tuple:
    """
    Prétraiter les données en soustrayant la note moyenne pour chaque film (chaque ligne).
    Inclure uniquement les notes réelles : R(i,j)=1.

    [Ynorm, Ymean] = normalizeRatings(Y, R) normalise Y pour que chaque film
    ait une note de 0 en moyenne. Les films non notés ont alors une note moyenne (0)
    Renvoie la note moyenne dans Ymean.

    Paramètres:
    ----------
    Y : np.ndarray
        Le tableau de notes à normaliser.
    R : np.ndarray
        Le tableau indiquant si un utilisateur a noté un film.
    axis : int
        L'axe numpy à utiliser pour calculer la note moyenne.

    Retours:
    -------
    Tuple
        Le tableau normalisé des notes et la note moyenne.
    """
    Ymean = (np.sum(Y * R, axis=axis) / (np.sum(R, axis=axis) + 1e-12)).reshape(-1, 1)

    if axis == 0:
        Ynorm = Y.T - np.multiply(Ymean, R.T)
    else:
        Ynorm = Y - np.multiply(Ymean, R)

    return (Ynorm, Ymean)


def cofi_cost_func_v(X, W, W_features, b, Y, R,movie_features, lambda_):
    """
    Renvoie le coût pour le filtrage basé sur le contenu
    Vectorisé pour la vitesse. Utilise les opérations TensorFlow pour être compatible avec une boucle d'entraînement personnalisée.

    Paramètres:
    -----------
    X: np.ndarray (num_movies,num_features)
        Matrice des caractéristiques des items
    W: np.ndarray (num_users,num_features)
        Matrice des paramètres des utilisateurs
    W_features: np.ndarray (num_movie_features, num_features)
        Matrice de poids des features des films
    b: np.ndarray (1, num_users)
        Vecteur des paramètres des utilisateurs
    Y: np.ndarray (num_movies,num_users)
        Matrice des notes des utilisateurs pour les films
    R: np.ndarray (num_movies,num_users)
        Matrice où R(i, j) = 1 si le i-ème film a été noté par le j-ème utilisateur
    movie_features: np.ndarray
        Matrice des features des films
    lambda_: float
        Paramètre de régularisation

    Retours:
    -------
    float
        La valeur du coût étant donné les paramètres.
    """
    #Calcule le biais apportés par les caractéristiques (features) des films
    movie_bias = tf.matmul(movie_features, W_features)
    
    # On ajoute ce biais aux caractéristiques déjà connues (X) pour prendre en compte l'influence
    # des features sur les notes des utilisateurs (= ajouts de nouvelles caractéristiques) 
    j = (tf.linalg.matmul(X + movie_bias, tf.transpose(W)) + b - Y) * R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_ / 2) * (
        tf.reduce_sum(X**2) + tf.reduce_sum(W**2) + tf.reduce_sum(W_features**2)
    )
    return J

# Chargement et traitements des données

#### Chargement de movieLens (movies et ratings)

In [3]:
#Movies
df_movies1 = pd.read_csv(
        "./data/movielens_complete/ml-1m/movies.dat",
        sep="::",
        encoding='ISO-8859-1',
        engine='python',
        header=None
    )

df_movies1.columns = ["movieId", "title", "genres"]

#Séparation des titres en titre et années pour pouvoir le lier à l'autre dataset
df_movies1['title'], df_movies1['year'] = zip(*df_movies1['title'].map(extract_year_and_title))


#Ratings
df_ratings1 = pd.read_csv(
        "./data/movielens_complete/ml-1m/ratings.dat",
        sep="::",
        encoding='ISO-8859-1',
        engine='python',
        header=None
    )

df_ratings1.columns = ["userId","movieId","Rating","timestamp"]


#matrice d'interaction utilisateur-films
df_matrix = df_ratings1.pivot(index="movieId", columns="userId", values="Rating")

#### Chargement de IMbd (titleBasics, title_crew et ratings)

In [4]:
#Chargement de titleBasics
df_movies2 = pd.read_csv(
        "./data/IMDb/title.basics.tsv",
        sep='\t'
    )

#Surpression de tout ce qui n'est pas un movie pour pouvoir le combiner plus tard
df_movies2.drop(df_movies2[df_movies2['titleType'] != 'movie'].index, inplace=True)


#Chargement de titleCrew
df_crew = pd.read_csv(
        "./data/IMDb/title.crew.tsv",
        sep='\t'
    )

df_crew.drop(columns=['writers'], inplace=True)

#chargement de ratings
df_ratings2 = pd.read_csv(
        "./data/IMDb/title.ratings.tsv",
        sep='\t'
    )

/tmp/ipykernel_34/3002934010.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies2 = pd.read_csv(


#### Merge des deux datasets

In [5]:
#merge des datasets movies pour obtenir plus de metadata
df_movies = pd.merge(df_movies1, df_movies2, left_on=['title', 'year'], right_on=['primaryTitle', 'startYear'], how='left')
df_movies.drop_duplicates(subset='movieId', keep='first', inplace=True)
 
#merge des movies et de ratings pour avoir les notes moyennes en metadata
df_movies = pd.merge(df_movies, df_ratings2, on='tconst', how='left')

#merge de movies et de crew pour avoir les directors en metadata
df_movies = pd.merge(df_movies, df_crew, on='tconst', how='left')

#nettoyage du df_movies
df_movies['genres'] = df_movies.apply(combine_genres, axis=1)
df_movies['directors'] = df_movies['directors'].fillna('')
df_movies['isAdult'] = df_movies['isAdult'].fillna(0)
#pour ne pas mettre 0 et influencer les choix trop négativement pour ces films, on remplace les nan par la moyenne des averageRating
df_movies['averageRating'] = df_movies['averageRating'].fillna(df_movies['averageRating'].mean())
df_movies['averageRating'] = df_movies['averageRating'] / 2
df_movies.drop(columns=['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'startYear', 'endYear', 'runtimeMinutes', 'genres_x', 'genres_y'], inplace=True)

# Rajout des films qui n'ont jamais été notés pour permettre l'utilisation des features plus tard
movie_ids_in_movies = set(df_movies['movieId'])
movie_ids_in_ratings = set(df_matrix.index)
missing_movie_ids = movie_ids_in_movies - movie_ids_in_ratings
new_entries = pd.DataFrame(np.nan, index=list(missing_movie_ids), columns=df_matrix.columns)
df_matrix = pd.concat([df_matrix, new_entries], ignore_index=False)

/tmp/ipykernel_34/970431456.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_movies['isAdult'] = df_movies['isAdult'].fillna(0)


#### Création des np.array pour le collaborative filtering

In [6]:
R = df_matrix.notna().astype(int).to_numpy()
df_matrix = df_matrix.fillna(0)
Y = df_matrix.to_numpy()

# Features_engineering

- encodage des genres et des directors en matrice binaire pour être utilisés comme features
- utilisation de isAdult et de averageRating (remis sur 5) comme features
- scaling de la matrice de features pour donner le même poids à chacune d'entre elles et pour lui permettre d'être utilisée plus tard dans l'entraînement

Genres et directors sont choisis car les gens ont souvent des préférences pour des genres ou des préférences pour certains directeurs de films.

isAdult est choisi car certaines personnes peuvent préférer des films considérés comme pour les adultes plutôt que des films plus enfantins.

averageRating est choisi car c'est une donnée que les gens peuvent regarder pour choisir un film (si un film est bien noté en moyenne, on aura tendance à le préférer). Cela peut également aider si un film n'est pas beaucoup noté ou n'est pas du tout noté par les utilisateurs.

In [7]:
#transformation des strings en liste pour être utilisé par MultiLabelBinarizer
df_movies['genres'] = df_movies['genres'].apply(lambda x: x.split(','))
df_movies['directors'] = df_movies['directors'].apply(lambda x: x.split(','))

# Transformation en matrice binaire
genres_mlb = MultiLabelBinarizer()
genres_encoded = genres_mlb.fit_transform(df_movies['genres'])
genres_df = pd.DataFrame(genres_encoded, columns=genres_mlb.classes_)

directors_mlb = MultiLabelBinarizer()
directors_encoded = directors_mlb.fit_transform(df_movies['directors'])
directors_df = pd.DataFrame(directors_encoded, columns=directors_mlb.classes_)

features = pd.concat([genres_df,directors_df, df_movies[['isAdult', 'averageRating']]], axis=1)

# Scaling de la matrice de features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

movie_features = tf.constant(features_scaled, dtype=tf.float64)

In [8]:
#  Données utiles
num_movies, num_users = Y.shape
num_movie_features = movie_features.shape[1]
movieList = df_movies["title"].to_list()

#Séparation en train, val et test
Y_train, Y_val, Y_test = split_ratings(Y, val_size=627, test_size=626)
Ynorm, Ymean = normalizeRatings(Y_train, R)

# Modèle de recommendation

In [9]:
class MovieRecommender:

    def __init__(self, num_users, num_movies, num_features, num_movie_features, movieFeatures):
        """
        Initialisation de notre système de recommendation

        Paramètres:
        -----------
        num_user: int
            nombre d'utilisateurs dans notre système
        num_movies: int
            nombre de films
        num_features: int
            nombre de features du système
        num_movie_features: int
            nombre de caractèristiques (features type genre, directors etc) des films
        movieFeatures: tf.tensor
            tensor contenant les caractéristiques (features type genre, directors etc) pour chaque film
        """
        self.num_users = num_users
        self.num_movies = num_movies
        self.num_features = num_features
        self.num_movie_features = num_movie_features
        self.movieFeatures = movieFeatures
        
        tf.random.set_seed(1234)
        
        self.X = tf.Variable(tf.random.normal((self.num_movies, self.num_features), dtype=tf.float64), name="X")
        self.W = tf.Variable(tf.random.normal((self.num_users, self.num_features), dtype=tf.float64), name="W")
        self.b = tf.Variable(tf.random.normal((1, self.num_users), dtype=tf.float64), name="b")
        self.W_features = tf.Variable(tf.random.normal((self.num_movie_features, self.num_features), dtype=tf.float64), name="W_features")
        
        self.mean = None
        
        
    def fit(self, Y, mean, r_mask, iterations, lambda_, learning_r):
        """
        Fonction qui entraîne notre système de recommendation
        
        Paramètres:
        -----------
        Y: np.ndarray
            Le tableau de notes
        mean: np.array
            Le tableau avec les notes moyennes (pour calculer plus tard les résultats)
        r_mask: np.ndarray
            Le tableau indiquant si un utilisateur a noté un film.
        iterations: int
            Le nombre d'itérations à réaliser pour l'entrainement du modèle
        lambda_: float
            Le paramètre de régularisation
        learning_r: float
            Le learning_rate pour l'optimizer
        """
        self.mean = mean
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_r)
        for iter in range(iterations):
            with tf.GradientTape() as tape:
                # Calcule le coût
                cost_value = cofi_cost_func_v(self.X, self.W, self.W_features, self.b, Y, r_mask, self.movieFeatures, lambda_)

            grads = tape.gradient(cost_value, [self.X, self.W, self.W_features, self.b])

            # Effectue une étape de descente de gradient en mettant à jour la valeur des variables pour minimiser la perte.
            optimizer.apply_gradients(zip(grads, [self.X, self.W, self.W_features, self.b]))

            if iter % 20 == 0:
                print(f"Training loss at iteration {iter}: {cost_value:0.1f}")
    
    
    def getResults(self):
        """
        Fonction qui récupère la matrice de scores
        
        Retours:
        -------
        np.ndarray
            Notes des utilisateurs pour chaque film
        """
        movie_bias = tf.matmul(self.movieFeatures, self.W_features)
        p = np.matmul(self.X.numpy() + movie_bias.numpy(), np.transpose(self.W.numpy())) + self.b.numpy()

        # ajout de la moyenne pour obtenir les bonnes valeurs
        pm = p + self.mean
        
        # bloquer les valeurs entre 0 et 5 pour éviter des valeurs trop grandes (possible à cause de l'ajout de la moyenne)
        pm = np.clip(pm, 0, 5)
        
        return pm
    

    def getMetrics(self, Yval, movieList, printExemples=False):
        """
        Fonction qui récupère l'erreur moyenne des prédictions et la précision des prédictions pour évaluer le modèle
        
        Paramètres:
        ----------
        Yval: np.ndarray
            Le tableau des notes
        movieList: List
            La liste contenant les titres de tous les films
        printExemples: bool
            Variable qui permet de print certaines prédictions et leur véritable valeur
        
        Retours:
        -------
        Tuple(Float, Float)
            la précision du modèle et la racine carrée de l'erreur quadratique moyenne
        """
        pm = self.getResults()

        # Récupération des notes de départ
        non_zero_indices = np.nonzero(Yval)
        rows, cols = non_zero_indices
        
        if(printExemples):
            for k in range (5):
                my_ratings = Yval[:,cols[k]]
                my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

                my_predictions = pm[:, k]

                print("\n\nOriginal vs Predicted ratings for",(k + 1), "\n")
                for i in range(len(my_ratings)):
                    if my_ratings[i] > 0:
                        print(
                            f"Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {movieList[i]}"
                        )

        predicted_ratings = pm[rows, cols]
        real_ratings = Yval[rows, cols]

        # Calcul de la racine carrée de l'erreur quadratique moyenne (RMSE)
        # Ce métrique permet de se rendre compte de la capacité de notre modèle à prédire la bonne valeur
        # Il est également plus sensible aux grands écarts que la simple erreur absolue moyenne
        rmse = np.sqrt(np.mean((predicted_ratings - real_ratings) ** 2))
        
        # Seuil pour considérer une recommandation comme pertinente
        threshold = 4

        # Convertir les notes en recommandations pertinentes ou non pertinentes
        predicted_pertinent = predicted_ratings >= threshold
        real_pertinent = real_ratings >= threshold

        # Calcul de la précision (et donc la pertinence des propositions de notre modèle)
        precision = precision_score(real_pertinent, predicted_pertinent)
        
        return precision, rmse
        

    def getCoupleRatings(self, user_id1, user_id2):
        """
        Fonction qui permet de récupérer les notes données par un couple de personnes
        
        Paramètres:
        -----------
        user_id1: int
            Le premier utilisateur de notre couple
        user_id2: int
            Le deuxième utilisateur de notre couple
        
        Retours:
        --------
        np.array
            L'array contenant la note donnée par notre couple pour chaque film
        """
        pm = self.getResults()
        
        user_ratings1 = pm[:, user_id1-1]
        user_ratings2 = pm[:, user_id2-1]
        
        # La note d'un couple pour un film correspond à la moyenne de leurs notes respectives
        couple_ratings = (user_ratings1 + user_ratings2) / 2

        return couple_ratings
    

    def getMovieRecommendation(self, user_id1, user_id2, movies, R):
        """
        Fonction qui récupère le meilleur film pour un couple d'utilisateurs
        
        Paramètres:
        -----------
        user_id1: int
            Le premier utilisateur de notre couple
        user_id2: int
            Le deuxième utilisateur de notre couple
        movies: pd.dataframe
            Le dataframe contenant tous les films, leurs caractéristiques etc
        R: np.ndarray
            Le tableau indiquant si un utilisateur a noté un film.
            
        Retours:
        --------
        object:
            object contenant l'id, le titre et le score du film choisi pour ce couple
        """
        couple_ratings = self.getCoupleRatings(user_id1, user_id2)
        
        # Récupère les films vu par chaque utilisateur
        movies_seen1 = R[:, user_id1-1]
        movies_seen2 = R[:, user_id2-1]

        couple_scores = []
        fallback_scores = []

        # Calcule le couple score pour chaque film (la note donnée avec parfois un bonus)
        # Elimine les films vu par les 2 utilisateurs
        for i in range(len(couple_ratings)):
            avg_rating = couple_ratings[i]

            if movies_seen1[i] and movies_seen2[i]:
                # On garde sous la main ceux notés par les deux au cas où il n'y a aucun nouveau film
                fallback_scores.append((i, avg_rating))
                continue
            elif movies_seen1[i] or movies_seen2[i]:
                couple_score = avg_rating
            else:
                # Les films vu par aucun des deux ont un bonus pour les prioritiser
                couple_score = avg_rating + 1

            couple_scores.append((i, couple_score))

        # Choisi le film avec le meilleur couple_score
        if not couple_scores:
            best_movie = max(fallback_scores, key=lambda x: x[1])
        else:
            best_movie = max(couple_scores, key=lambda x: x[1])

        best_movie_id = best_movie[0]
        best_movie_score = best_movie[1]
        best_movie_title = movies.iloc[best_movie_id]['title']

        return {
            'movie_id': best_movie_id + 1,
            'title': best_movie_title,
            'couple_score': best_movie_score
        }
        

# Utilisation du modèle avec Optuna pour trouver les meilleurs hyperparamètres

Nous avons deux options pour optuna:
- minimiser le rmse (et donc obenir des notes prédies le plus proche possible de la réelle)
- maximiser la précision (et donc s'assurer que les prédictions données soient cohérentes avec ce les utilisateurs trouvent bien)

Minimiser les erreurs permettrait d'avoir des notes plus proches de la réalité et donc de pouvoir dans certains cas obtenir une meilleure précision (sauf si les notes sont toujours juste en dessous du seuil). Nous allons donc essayer de minimiser la rmse.

In [10]:
def objective(trial):
    # Définition des hyperparamètres à optimiser
    # Les valeurs bornes ont été trouvées après plusieurs essais avec optuna
    num_features = trial.suggest_int('num_features', 50, 250)
    lambda_ = trial.suggest_float('lambda', 1e0, 1e1, log=True)
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True)
    iterations = trial.suggest_int('iterations', 200, 500)

    # Création et entraînement du modèle
    # Utilisation de R * (Y_train > 0) pour séparer totalement le train set du reste et ne prendre en compte
    # que les films notés du train set
    recommender = MovieRecommender(num_users, num_movies, num_features, num_movie_features, movie_features)
    recommender.fit(Ynorm, Ymean, R * (Y_train > 0), iterations, lambda_, learning_rate)

    # Évaluation du modèle sur le set de validation
    precision, rmse = recommender.getMetrics(Y_val, movieList)
    
    print("Précision du modèle: ", precision)
    
    return rmse

# Création d'une étude Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

best_params = study.best_params

# Affichage des meilleurs résultats
print("Meilleurs hyperparamètres trouvés: ", best_params)
print("Meilleure RMSE sur le set de validation: ", study.best_value)

# Visualisation de l'importance des hyperparamètres
optuna.visualization.plot_param_importances(study)

[I 2024-07-05 13:38:54,207] A new study created in memory with name: no-name-ffcaa807-147e-44c3-ba2a-0f9444ea2bae


Training loss at iteration 0: 66544636687.9
Training loss at iteration 20: 13596779869.0
Training loss at iteration 40: 4182288907.6
Training loss at iteration 60: 1830114605.0
Training loss at iteration 80: 981811856.0
Training loss at iteration 100: 590712622.8
Training loss at iteration 120: 382411532.5
Training loss at iteration 140: 261228498.3
Training loss at iteration 160: 186218594.7
Training loss at iteration 180: 137482925.1
Training loss at iteration 200: 104526785.0
Training loss at iteration 220: 81474591.0
Training loss at iteration 240: 64875237.3
Training loss at iteration 260: 52618094.7
Training loss at iteration 280: 43366515.5
Training loss at iteration 300: 36247736.9


[I 2024-07-05 13:41:24,642] Trial 0 finished with value: 2.370345652096164 and parameters: {'num_features': 86, 'lambda': 2.504176848173028, 'learning_rate': 0.019326437887489126, 'iterations': 308}. Best is trial 0 with value: 2.370345652096164.


Précision du modèle:  0.6164383561643836
Training loss at iteration 0: 89317740964.6
Training loss at iteration 20: 1285896096.2
Training loss at iteration 40: 176134399.0
Training loss at iteration 60: 41744902.9
Training loss at iteration 80: 17700403.1
Training loss at iteration 100: 10842998.3
Training loss at iteration 120: 7769918.2
Training loss at iteration 140: 5977305.4
Training loss at iteration 160: 4800593.0
Training loss at iteration 180: 3979492.2
Training loss at iteration 200: 3383543.7
Training loss at iteration 220: 2937637.8
Training loss at iteration 240: 2595618.3
Training loss at iteration 260: 2327748.5
Training loss at iteration 280: 2114219.7
Training loss at iteration 300: 1941131.0
Training loss at iteration 320: 1799160.4


[I 2024-07-05 13:43:59,950] Trial 1 finished with value: 1.945836738564395 and parameters: {'num_features': 117, 'lambda': 1.1733250679702198, 'learning_rate': 0.08744295104090717, 'iterations': 321}. Best is trial 1 with value: 1.945836738564395.


...

Précision du modèle:  0.6014234875444839
Training loss at iteration 0: 38612235355.6
Training loss at iteration 20: 26891456381.4
Training loss at iteration 40: 19243031962.5
Training loss at iteration 60: 14230141065.7
Training loss at iteration 80: 10827408752.3
Training loss at iteration 100: 8426604561.3
Training loss at iteration 120: 6675844908.0
Training loss at iteration 140: 5364326773.4
Training loss at iteration 160: 4360043071.1
Training loss at iteration 180: 3577024150.2
Training loss at iteration 200: 2957396370.6
Training loss at iteration 220: 2461061371.3
Training loss at iteration 240: 2059506334.4
Training loss at iteration 260: 1731968690.6
Training loss at iteration 280: 1462997879.9
Training loss at iteration 300: 1240871227.1
Training loss at iteration 320: 1056543379.9
Training loss at iteration 340: 902934709.7
Training loss at iteration 360: 774438486.0


[I 2024-07-05 15:11:00,485] Trial 28 finished with value: 2.677864180627297 and parameters: {'num_features': 50, 'lambda': 9.913061881064628, 'learning_rate': 0.004535356239760774, 'iterations': 373}. Best is trial 15 with value: 1.2912629249811782.


Précision du modèle:  0.5784313725490197
Training loss at iteration 0: 67215373404.1
Training loss at iteration 20: 13343386632.1
Training loss at iteration 40: 4035960824.2
Training loss at iteration 60: 1755863079.5
Training loss at iteration 80: 942315186.7
Training loss at iteration 100: 568464708.1
Training loss at iteration 120: 369342753.1
Training loss at iteration 140: 253285726.2
Training loss at iteration 160: 181238274.4
Training loss at iteration 180: 134259946.9
Training loss at iteration 200: 102368797.0
Training loss at iteration 220: 79974602.5
Training loss at iteration 240: 63789429.2
Training loss at iteration 260: 51797732.4
Training loss at iteration 280: 42719152.6
Training loss at iteration 300: 35714761.6
Training loss at iteration 320: 30219720.1
Training loss at iteration 340: 25844461.2
Training loss at iteration 360: 22314464.7
Training loss at iteration 380: 19432508.1
Training loss at iteration 400: 17054407.2


[I 2024-07-05 15:14:26,724] Trial 29 finished with value: 2.3919656494948973 and parameters: {'num_features': 88, 'lambda': 1.253498834583448, 'learning_rate': 0.019635238304385672, 'iterations': 420}. Best is trial 15 with value: 1.2912629249811782.


Précision du modèle:  0.6107142857142858
Meilleurs hyperparamètres trouvés:  {'num_features': 52, 'lambda': 1.5200780975136425, 'learning_rate': 0.09724992292230974, 'iterations': 439}
Meilleure RMSE sur le set de validation:  1.2912629249811782


# Résultats

In [11]:
# Utilisation des meilleurs paramètres pour entrainer notre modèle
recommender_final = MovieRecommender(num_users, num_movies, best_params['num_features'], num_movie_features, movie_features)
recommender_final.fit(Ynorm, Ymean, R * (Y_train >0), best_params['iterations'], best_params['lambda'], best_params['learning_rate'])

# Obtention de l'erreur moyenne de notre modèle sur le validation set et le test set pour visualiser ses performances
precision_val, rmse_val = recommender_final.getMetrics(Y_val, movieList)
precision_test, rmse_test = recommender_final.getMetrics(Y_test, movieList)

print(f"\n\nPrécision sur val set : {precision_val}")
print(f"\nRMSE sur val set : {rmse_val}")
print(f"\n\nPrécision sur test set : {precision_test}")
print(f"\nRMSE sur test set : {rmse_test}")

Training loss at iteration 0: 39738856463.9
Training loss at iteration 20: 466642933.2
Training loss at iteration 40: 66511114.1
Training loss at iteration 60: 14723587.6
Training loss at iteration 80: 5893748.2
Training loss at iteration 100: 3561811.0
Training loss at iteration 120: 2589403.1
Training loss at iteration 140: 2047646.6
Training loss at iteration 160: 1701846.7
Training loss at iteration 180: 1465218.3
Training loss at iteration 200: 1295856.4
Training loss at iteration 220: 1170428.2
Training loss at iteration 240: 1074900.2
Training loss at iteration 260: 1000396.5
Training loss at iteration 280: 941084.6
Training loss at iteration 300: 893009.6
Training loss at iteration 320: 853416.1
Training loss at iteration 340: 820339.7
Training loss at iteration 360: 792350.2
Training loss at iteration 380: 768387.7
Training loss at iteration 400: 747653.4
Training loss at iteration 420: 729536.8


Précision sur val set : 0.7230769230769231


RMSE sur val set : 1.29126292498117

In [12]:
#Recommendation pour les utilisateurs 1 et 2
best_movie = recommender_final.getMovieRecommendation(1,2, df_movies, R)
best_movie

{'movie_id': 13, 'title': 'Balto', 'couple_score': 6.0}